## Agentic AI Framework Project

#### This notebook will showcase the follwoing:

* Agent workflow
* Use of tools to call functions
* Agent collaboration via Tools and Handoffs

In [1]:
import asyncio
from dotenv import load_dotenv
from agents import Agent, trace, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

### Step1: Agent Workflow

In [3]:
instructions1 = """You are a sales agent working for InsightStat, which is a company offering data science solutions. This \
company leverages the latest AI tools to design tailored data science solutions. Your job is to write cold emails that are serious \
and professional
"""

instructions2 = """You are a humorous, engaging sales agent working for InsightStat, which is a company offering data science solutions. This \
company leverages the latest AI tools to design tailored data science solutions. Your job is to write witty, engaging cold emails will likely get you \
a response"""

instructions3 = """You are a busy ales agent working for InsightStat, which is a company offering data science solutions. This
company leverages the latest AI tools to design tailored data science solutions. Your job is to write serious cold emails that are concise
and to the point"""

In [4]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = 'gpt-4o-mini'
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = 'gpt-4o-mini'
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = 'gpt-4o-mini'
)

In [5]:
result = Runner.run_streamed(sales_agent1, input= "Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end = "", flush=True)

Subject: Unlock the Power of Data with Tailored Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out to introduce you to InsightStat, where we specialize in delivering bespoke data science solutions that harness the latest advancements in AI.

In today’s data-driven landscape, making informed decisions is crucial. Our team at InsightStat collaborates with organizations like yours to analyze and extract valuable insights from complex datasets. We have a proven track record of enhancing operational efficiency, improving customer experience, and enabling strategic growth.

I would love the opportunity to discuss how our tailored solutions can address your specific challenges and drive actionable insights within your organization. Would you be open to a brief call next week to explore this further?

Thank you for considering this opportunity. I look forward to the possibility of working together.

Best regards,

[Your Name] 

In [6]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output, "\n\n")

Subject: Unlock the Power of Data with Tailored AI Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am with InsightStat, where we specialize in harnessing the power of data science to drive measurable results for businesses like yours.

In today’s competitive landscape, leveraging data effectively can be the key to unlocking new growth opportunities. At InsightStat, we utilize cutting-edge AI tools to design customized data science solutions that align with your organization's unique goals and challenges.

We have worked successfully with companies across various industries, helping them to:
- Enhance decision-making through predictive analytics
- Optimize processes using machine learning models
- Uncover valuable insights from complex datasets

I would love the opportunity to discuss how we can support your business in achieving its strategic objectives. Would you be available for a brief call next week to explore this further?

Th

In [7]:
sales_picker = Agent(
    name = "sales_picker",
    instructions = "You have to pick the best cold sales email from the given oprions \
        Imagine you are a customer who will respond to one of the emails that you liked the best \
        Do not give any explanation. Just return the best email",
        model= "gpt-4o-mini" 
)

In [8]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

emails = "Cold sales emails:\n\n" + "\n\nEmail: \n\n".join(outputs)

best = await Runner.run(sales_picker, emails)

print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Is Your Data Feeling a Little Shy? 🤔📊

Hey [Recipient's Name],

Ever feel like your data is hiding at a party, sipping a soda in the corner while everyone else is dancing? 🎉 Well, that’s where we come in!

At InsightStat, we specialize in coaxing those bashful numbers onto the dance floor with our tailored data science solutions. We know a thing or two (or three... or four) about using the latest AI tools to transform your data from wallflower to the life of the party!

Imagine your metrics busting out some sweet moves, giving you the insights you need to make informed decisions and impress your boss (bonus: who doesn’t love great party tricks?). 🚀

Are you ready to turn your data from shy to savvy? Let’s chat — I promise I won't make you share any embarrassing dance stories... unless you really want to!

Looking forward to hearing from you!

Best,  
[Your Name]  
[Your Job Title]  
InsightStat  
[Your Phone Number]  
[Your LinkedIn Profile]  

P.S. Don’t let

### Use of Tools

In [9]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = 'gpt-4o-mini'
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = 'gpt-4o-mini'
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = 'gpt-4o-mini'
)

In [10]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for InsightStat, which is a company offering data science solutions. This company leverages the latest AI tools to design tailored data science solutions. Your job is to write cold emails that are serious and professional\n', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [ ]:
@function_tool
def send_email(body:str):
    """ Send out an email with the given body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ['SENDGRID_API_KEY'])
    from_email = Email("kausthab.phukan@gmail.com")
    to_email = To("kausthab.phukan@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content)
    sg.send(mail)
    return {"status":"success"}

In [12]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001E664B811C0>, strict_json_schema=True, is_enabled=True)

Converting Agent into a tool

In [13]:
tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description= "Write a cold sales email")

tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001E6670F3CE0>, strict_json_schema=True, is_enabled=True)

If we can do it for 1, we can do it for all the others

In [14]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description= description)
tool2 = sales_agent2.as_tool(tool_name= "sales_agent2", tool_description= description)
tool3 = sales_agent3.as_tool(tool_name= "sales_agent3", tool_description= description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001E6670F3920>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001E666B4AFC0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

### Creating the Sales Manager that will select the best email to send

In [15]:
instructions = """You are Sales Manager at Insight Stat and responsible for looking over all the communications your sales agents send out to their prospects. \
    It is also your responsibility to select the best cold sales email using the sales agent tool. And this is how you should go about doing it:

    1.  Use the sales_agent1, sales_agent2, and sales_agent3 tools to generate three different email drafts. Call each tool once to get one draft from each. \
    Do not proceed until all the drafts have been prepared and are ready.


    2. Evaluate and draft selection: Evaluate these 3 drafts and select the best/effective amongst these three. You do not need to give any explanation.

    3.  Use the send_email tool to send the selected best email (only the best email) to the prospect 

    Critical rule:

    - You must use the  sales_agent1, sales_agent2, and sales_agent3 tools to write the emails, never write the emails yourself

    - You will use the send_email tool to send only one email to the prospect, and never send more than one email
"""

sales_manager = Agent(name= "sales_manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "send a cold email to a prospect addressing 'Dear CEO' "

with trace("sales manager"):
    result = await Runner.run(sales_manager, message)

## Handoffs